Club Mahindra (Club M) makes significant revenue from Food and Beverages (F&B) sales in their resorts. The members of Club M are offered a wide variety of items in either buffet or À la carte form. Following are some benefits that the model to predict the spend by a member in their next visit to a resort will bring:
Predicting the F&B spend of a member in a resort would help in improving the pre-sales during resort booking through web and mobile app
Targeted campaigns to suit the member taste and preference of F&B
Providing members in the resort with a customized experience and offers
Help resort kitchen to plan the inventory and food quantity to be prepared in advance
Given the information related to resort, club member, reservation etc. the task is to predict average spend per room night on food and beverages for the each reservation in the test set.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
train = pd.read_csv('train.csv')

In [3]:
test = pd.read_csv('test.csv')

In [4]:
print(pd.get_option("display.max_columns"))


20


In [5]:
pd.set_option('display.max_columns', None)


In [6]:
train.head()

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,amount_spent_per_room_night_scaled
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,05/04/18,05/04/18,06/04/18,3,1,2,0,46,3,3,3,1,2.0,7.0,3,3,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,C,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,7.706428
1,03930f033646d073462b35d411616323597715ac4fc398...,23/01/15,11/04/15,16/04/15,1,1,2,0,46,3,3,4,5,2.0,7.0,5,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,A,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,6.662563
2,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,28/01/15,01/02/15,05/02/15,1,1,2,0,47,1,5,4,4,2.0,7.0,1,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,E,A,535fa30d7e25dd8a49f1536779734ec8286108d115da50...,7.871602
3,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04...,02/05/15,11/06/15,16/06/15,1,1,2,2,46,2,2,3,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,5.344943
4,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb29...,02/09/15,14/12/15,19/12/15,1,1,2,0,46,2,2,4,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,7.059346


In [7]:
print(train.shape)
print(test.shape)

(341424, 24)
(146765, 23)


In [8]:
train.columns

Index(['reservation_id', 'booking_date', 'checkin_date', 'checkout_date',
       'channel_code', 'main_product_code', 'numberofadults',
       'numberofchildren', 'persontravellingid', 'resort_region_code',
       'resort_type_code', 'room_type_booked_code', 'roomnights',
       'season_holidayed_code', 'state_code_residence', 'state_code_resort',
       'total_pax', 'member_age_buckets', 'booking_type_code', 'memberid',
       'cluster_code', 'reservationstatusid_code', 'resort_id',
       'amount_spent_per_room_night_scaled'],
      dtype='object')

In [9]:
test.columns

Index(['reservation_id', 'booking_date', 'checkin_date', 'checkout_date',
       'channel_code', 'main_product_code', 'numberofadults',
       'numberofchildren', 'persontravellingid', 'resort_region_code',
       'resort_type_code', 'room_type_booked_code', 'roomnights',
       'season_holidayed_code', 'state_code_residence', 'state_code_resort',
       'total_pax', 'member_age_buckets', 'booking_type_code', 'memberid',
       'cluster_code', 'reservationstatusid_code', 'resort_id'],
      dtype='object')

In [10]:
#Concatenating the train and test data
df=train.append(test,ignore_index=True)


D:\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [11]:
df.shape

(488189, 24)

In [12]:
df.describe()

,amount_spent_per_room_night_scaled,booking_type_code,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax
count,341424.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488189.000000,488040.000000,481165.000000,488189.000000,488189.000000
mean,7.720801,1.123952,1.902587,2.207459,3.275522,0.362573,209.955489,1.773303,2.194687,2.889850,3.735949,2.676035,7.258755,4.759472,3.191893
std,1.090109,0.329527,0.915112,0.821989,1.764458,0.758078,864.891408,0.751550,1.549417,0.804194,2.480057,0.884669,4.717773,3.103048,1.166638
min,1.600397,1.000000,1.000000,1.000000,0.000000,0.000000,45.000000,1.000000,0.000000,1.000000,-45.000000,1.000000,1.000000,1.000000,0.000000
25%,7.187260,1.000000,1.000000,2.000000,2.000000,0.000000,45.000000,1.000000,1.000000,2.000000,2.000000,2.000000,4.000000,2.000000,2.000000
50%,7.860814,1.000000,2.000000,2.000000,3.000000,0.000000,45.000000,2.000000,2.000000,3.000000,3.000000,3.000000,8.000000,5.000000,3.000000
75%,8.419144,1.000000,3.000000,3.000000,4.000000,0.000000,45.000000,2.000000,3.000000,3.000000,4.000000,3.000000,8.000000,7.000000,4.000000
max,10.816652,2.000000,3.000000,7.000000,32.000000,13.000000,4995.000000,3.000000,7.000000,6.000000,80.000000,4.000000,38.000000,13.000000,24.000000


In [13]:
#Ignore the target Variable season_holidayed_code and state_code_residence have some null values       

df.isnull().sum()/len(df)

amount_spent_per_room_night_scaled    0.300632
booking_date                          0.000000
booking_type_code                     0.000000
channel_code                          0.000000
checkin_date                          0.000000
checkout_date                         0.000000
cluster_code                          0.000000
main_product_code                     0.000000
member_age_buckets                    0.000000
memberid                              0.000000
numberofadults                        0.000000
numberofchildren                      0.000000
persontravellingid                    0.000000
reservation_id                        0.000000
reservationstatusid_code              0.000000
resort_id                             0.000000
resort_region_code                    0.000000
resort_type_code                      0.000000
room_type_booked_code                 0.000000
roomnights                            0.000000
season_holidayed_code                 0.000305
state_code_re

In [14]:
df.head(5)

,amount_spent_per_room_night_scaled,booking_date,booking_type_code,channel_code,checkin_date,checkout_date,cluster_code,main_product_code,member_age_buckets,memberid,numberofadults,numberofchildren,persontravellingid,reservation_id,reservationstatusid_code,resort_id,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax
0,7.706428,05/04/18,1,3,05/04/18,06/04/18,F,1,F,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,46,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,C,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,3,3,3,1,2.0,7.0,3,3
1,6.662563,23/01/15,1,1,11/04/15,16/04/15,F,1,F,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,46,03930f033646d073462b35d411616323597715ac4fc398...,A,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,3,3,4,5,2.0,7.0,5,2
2,7.871602,28/01/15,1,1,01/02/15,05/02/15,E,1,F,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,47,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,A,535fa30d7e25dd8a49f1536779734ec8286108d115da50...,1,5,4,4,2.0,7.0,1,2
3,5.344943,02/05/15,1,1,11/06/15,16/06/15,D,1,F,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,2,46,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04...,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,2,2,3,5,2.0,7.0,2,2
4,7.059346,02/09/15,1,1,14/12/15,19/12/15,D,1,F,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,46,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb29...,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,2,2,4,5,2.0,7.0,2,2


In [15]:
df.dtypes

amount_spent_per_room_night_scaled    float64
booking_date                           object
booking_type_code                       int64
channel_code                            int64
checkin_date                           object
checkout_date                          object
cluster_code                           object
main_product_code                       int64
member_age_buckets                     object
memberid                               object
numberofadults                          int64
numberofchildren                        int64
persontravellingid                      int64
reservation_id                         object
reservationstatusid_code               object
resort_id                              object
resort_region_code                      int64
resort_type_code                        int64
room_type_booked_code                   int64
roomnights                              int64
season_holidayed_code                 float64
state_code_residence              

In [16]:
df['booking_date'].head()

0    05/04/18
1    23/01/15
2    28/01/15
3    02/05/15
4    02/09/15
Name: booking_date, dtype: object

In [17]:
df.checkin_date.dtypes

dtype('O')

In [18]:
df.head()

,amount_spent_per_room_night_scaled,booking_date,booking_type_code,channel_code,checkin_date,checkout_date,cluster_code,main_product_code,member_age_buckets,memberid,numberofadults,numberofchildren,persontravellingid,reservation_id,reservationstatusid_code,resort_id,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax
0,7.706428,05/04/18,1,3,05/04/18,06/04/18,F,1,F,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,46,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,C,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,3,3,3,1,2.0,7.0,3,3
1,6.662563,23/01/15,1,1,11/04/15,16/04/15,F,1,F,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,46,03930f033646d073462b35d411616323597715ac4fc398...,A,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,3,3,4,5,2.0,7.0,5,2
2,7.871602,28/01/15,1,1,01/02/15,05/02/15,E,1,F,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,47,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,A,535fa30d7e25dd8a49f1536779734ec8286108d115da50...,1,5,4,4,2.0,7.0,1,2
3,5.344943,02/05/15,1,1,11/06/15,16/06/15,D,1,F,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,2,46,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04...,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,2,2,3,5,2.0,7.0,2,2
4,7.059346,02/09/15,1,1,14/12/15,19/12/15,D,1,F,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,46,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb29...,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,2,2,4,5,2.0,7.0,2,2


In [19]:
df['booking_date']=pd.to_datetime(df['booking_date'],format='%d/%m/%y')
df['checkin_date']=pd.to_datetime(df['checkin_date'], format='%d/%m/%y')
df['checkout_date']=pd.to_datetime(df['checkout_date'], format='%d/%m/%y')

In [54]:
#Stay Duration
df['Stay_duration']=df['checkout_date']-df['checkin_date']
#How early did he book the ticket?
df['Prior_chekin']=df['checkin_date']-df['booking_date']
#Till when does he intead to stay after booking
df['Prior_checkout']=df['checkout_date']-df['booking_date']

In [55]:
df['Stay_duration'].min()

Timedelta('1 days 00:00:00')

In [56]:
df['Stay_duration']=df['Stay_duration'].astype(str)
df['Stay_duration'] = df['Stay_duration'].str.replace(" days 00:00:00","")
df['Stay_duration']=pd.to_numeric(df['Stay_duration'])

In [57]:
df[df['Prior_chekin']<'0 days']

,amount_spent_per_room_night_scaled,booking_date,booking_type_code,channel_code,checkin_date,checkout_date,cluster_code,main_product_code,member_age_buckets,memberid,numberofadults,numberofchildren,persontravellingid,reservation_id,reservationstatusid_code,resort_id,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,Stay_duration,Prior_chekin,Prior_checkout
341465,NaN,2018-04-06,1,3,2012-04-20,2012-04-24,D,3,H,4e13e5c5482a6eba4e7badc21204f2add76b7203efe407...,2,0,4752,6ec2dd92d278aa97f8ab095ac0c0d18a7e9f18753c51c7...,C,98a3ab7c340e8a033e7b37b6ef9428751581760af67bba...,2,2,2,2,3.0,8.0,2,4,4.0,-2177 days,-2173 days
341481,NaN,2018-04-06,2,3,2012-04-28,2012-05-01,F,1,E,da2840cf5dd988b0641f48758a164621c36c26f7c48703...,3,0,45,54bf09a310761d2f874666557e3706012d85a47b14d0b8...,C,e7f6c011776e8db7cd330b54174fd76f7d0216b612387a...,1,1,3,7,2.0,8.0,4,3,3.0,-2169 days,-2166 days


In [58]:
df.shape

(488177, 27)

In [59]:
realtrain=df[df['amount_spent_per_room_night_scaled'].isnull()!=True]
realtest=df[df['amount_spent_per_room_night_scaled'].isnull()==True]

In [60]:
df=df.drop([15,34,60,79,81,113,116,129,130,147,149,155])

In [61]:
df.iloc[341465:341470,:]

,amount_spent_per_room_night_scaled,booking_date,booking_type_code,channel_code,checkin_date,checkout_date,cluster_code,main_product_code,member_age_buckets,memberid,numberofadults,numberofchildren,persontravellingid,reservation_id,reservationstatusid_code,resort_id,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,Stay_duration,Prior_chekin,Prior_checkout
341477,NaN,2014-10-08,1,3,2015-01-23,2015-01-26,E,2,F,13f58f100f8c666b33633cafc3c0b0eb2ce53dc438854b...,2,0,46,551cdf598c652a652a26e3d00d5304a364e764f253b6b3...,A,c75cb66ae28d8ebc6eded002c28a8ba0d06d3a78c6b5cb...,1,5,2,3,3.0,8.0,1,2,3.0,107 days,110 days
341478,NaN,2016-09-27,1,3,2016-10-09,2016-10-11,C,2,F,13f58f100f8c666b33633cafc3c0b0eb2ce53dc438854b...,4,0,45,59bb18224d6e0a0bea2380f86dc2fb8b783606a3e9e88d...,A,9400f1b21cb527d7fa3d3eabba93557a18ebe7a2ca4e47...,2,1,2,2,1.0,8.0,7,6,2.0,12 days,14 days
341479,NaN,2017-04-18,1,1,2017-06-14,2017-06-17,D,2,F,13f58f100f8c666b33633cafc3c0b0eb2ce53dc438854b...,2,0,45,d40383f0fd32392c5c6c84638c06fa753a6dd71ad032cb...,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,2,2,3,3,2.0,8.0,2,2,3.0,57 days,60 days
341480,NaN,2018-04-07,1,3,2018-04-07,2018-04-08,C,2,F,13f58f100f8c666b33633cafc3c0b0eb2ce53dc438854b...,2,0,46,854d31d8f81aec638df3c1d1c1b8916bfae0c017bf2cda...,A,da4ea2a5506f2693eae190d9360a1f31793c98a1adade5...,2,1,4,1,2.0,8.0,7,2,1.0,0 days,1 days
341481,NaN,2018-04-06,2,3,2012-04-28,2012-05-01,F,1,E,da2840cf5dd988b0641f48758a164621c36c26f7c48703...,3,0,45,54bf09a310761d2f874666557e3706012d85a47b14d0b8...,C,e7f6c011776e8db7cd330b54174fd76f7d0216b612387a...,1,1,3,7,2.0,8.0,4,3,3.0,-2169 days,-2166 days


In [62]:
df[df['Prior_checkout']<'0 days']


,amount_spent_per_room_night_scaled,booking_date,booking_type_code,channel_code,checkin_date,checkout_date,cluster_code,main_product_code,member_age_buckets,memberid,numberofadults,numberofchildren,persontravellingid,reservation_id,reservationstatusid_code,resort_id,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,Stay_duration,Prior_chekin,Prior_checkout
341465,NaN,2018-04-06,1,3,2012-04-20,2012-04-24,D,3,H,4e13e5c5482a6eba4e7badc21204f2add76b7203efe407...,2,0,4752,6ec2dd92d278aa97f8ab095ac0c0d18a7e9f18753c51c7...,C,98a3ab7c340e8a033e7b37b6ef9428751581760af67bba...,2,2,2,2,3.0,8.0,2,4,4.0,-2177 days,-2173 days
341481,NaN,2018-04-06,2,3,2012-04-28,2012-05-01,F,1,E,da2840cf5dd988b0641f48758a164621c36c26f7c48703...,3,0,45,54bf09a310761d2f874666557e3706012d85a47b14d0b8...,C,e7f6c011776e8db7cd330b54174fd76f7d0216b612387a...,1,1,3,7,2.0,8.0,4,3,3.0,-2169 days,-2166 days


In [44]:
df['Prior_chekin'].min()

Timedelta('-2177 days +00:00:00')

In [65]:
df['Prior_checkout']=df['Prior_checkout'].astype(str)
df['Prior_checkout'] = df['Prior_checkout'].str.replace("days","")
df['Prior_checkout']=pd.to_numeric(df['Prior_checkout'],errors='coerce')

In [66]:
df['Prior_checkout'].min()

1.0

In [67]:
df['Prior_checkout'].max()

180.0

In [68]:
df['Prior_chekin'].head()

0     0 days
1    78 days
2     4 days
3    40 days
4   103 days
Name: Prior_chekin, dtype: timedelta64[ns]

In [71]:
df['Prior_chekin']=df['Prior_chekin'].astype(str)
df['Prior_chekin'] = df['Prior_chekin'].str.replace("  00:00:00.000000000","")
df['Prior_chekin']=pd.to_numeric(df['Prior_chekin'],errors='coerce')

In [72]:
df.head()

,amount_spent_per_room_night_scaled,booking_date,booking_type_code,channel_code,checkin_date,checkout_date,cluster_code,main_product_code,member_age_buckets,memberid,numberofadults,numberofchildren,persontravellingid,reservation_id,reservationstatusid_code,resort_id,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,Stay_duration,Prior_chekin,Prior_checkout
0,7.706428,2018-04-05,1,3,2018-04-05,2018-04-06,F,1,F,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,46,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,C,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,3,3,3,1,2.0,7.0,3,3,1.0,0.0,1.0
1,6.662563,2015-01-23,1,1,2015-04-11,2015-04-16,F,1,F,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,46,03930f033646d073462b35d411616323597715ac4fc398...,A,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,3,3,4,5,2.0,7.0,5,2,5.0,78.0,83.0
2,7.871602,2015-01-28,1,1,2015-02-01,2015-02-05,E,1,F,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,47,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,A,535fa30d7e25dd8a49f1536779734ec8286108d115da50...,1,5,4,4,2.0,7.0,1,2,4.0,4.0,8.0
3,5.344943,2015-05-02,1,1,2015-06-11,2015-06-16,D,1,F,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,2,46,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04...,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,2,2,3,5,2.0,7.0,2,2,5.0,40.0,45.0
4,7.059346,2015-09-02,1,1,2015-12-14,2015-12-19,D,1,F,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,46,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb29...,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,2,2,4,5,2.0,7.0,2,2,5.0,103.0,108.0


In [73]:
df['Prior_checkout'] = df['Prior_checkout'].replace("-2173 days",0)
df['Prior_checkout'] = df['Prior_checkout'].replace("-2166",0)
df['Prior_chekin'] = df['Prior_chekin'].fillna(0)


In [74]:
df.shape

(488177, 27)

In [75]:
df['stay_divide_total']=df['total_pax']/df['Stay_duration']

In [76]:
df['booking_date_y']=df['booking_date'].dt.year
df['booking_date_m']=df['booking_date'].dt.month
df['booking_date_d']=df['booking_date'].dt.day
df['booking_date_w']=df['booking_date'].dt.week
df['checkin_date_y']=df['checkin_date'].dt.year
df['checkin_date_m']=df['checkin_date'].dt.month
df['checkin_date_d']=df['checkin_date'].dt.day
df['checkin_date_w']=df['checkin_date'].dt.week
df['checkout_date_y']=df['checkout_date'].dt.year
df['checkout_date_m']=df['checkout_date'].dt.month
df['checkout_date_d']=df['checkout_date'].dt.day
df['checkout_date_w']=df['checkout_date'].dt.week


In [77]:
df['stay_divide_total'].min()

0.0

In [78]:
df.columns

Index(['amount_spent_per_room_night_scaled', 'booking_date',
       'booking_type_code', 'channel_code', 'checkin_date', 'checkout_date',
       'cluster_code', 'main_product_code', 'member_age_buckets', 'memberid',
       'numberofadults', 'numberofchildren', 'persontravellingid',
       'reservation_id', 'reservationstatusid_code', 'resort_id',
       'resort_region_code', 'resort_type_code', 'room_type_booked_code',
       'roomnights', 'season_holidayed_code', 'state_code_residence',
       'state_code_resort', 'total_pax', 'Stay_duration', 'Prior_chekin',
       'Prior_checkout', 'stay_divide_total', 'booking_date_y',
       'booking_date_m', 'booking_date_d', 'booking_date_w', 'checkin_date_y',
       'checkin_date_m', 'checkin_date_d', 'checkin_date_w', 'checkout_date_y',
       'checkout_date_m', 'checkout_date_d', 'checkout_date_w'],
      dtype='object')

In [79]:
df.dtypes

amount_spent_per_room_night_scaled           float64
booking_date                          datetime64[ns]
booking_type_code                              int64
channel_code                                   int64
checkin_date                          datetime64[ns]
checkout_date                         datetime64[ns]
cluster_code                                  object
main_product_code                              int64
member_age_buckets                            object
memberid                                      object
numberofadults                                 int64
numberofchildren                               int64
persontravellingid                             int64
reservation_id                                object
reservationstatusid_code                      object
resort_id                                     object
resort_region_code                             int64
resort_type_code                               int64
room_type_booked_code                         

In [80]:
df.isnull().sum()

amount_spent_per_room_night_scaled    146765
booking_date                               0
booking_type_code                          0
channel_code                               0
checkin_date                               0
checkout_date                              0
cluster_code                               0
main_product_code                          0
member_age_buckets                         0
memberid                                   0
numberofadults                             0
numberofchildren                           0
persontravellingid                         0
reservation_id                             0
reservationstatusid_code                   0
resort_id                                  0
resort_region_code                         0
resort_type_code                           0
room_type_booked_code                      0
roomnights                                 0
season_holidayed_code                    149
state_code_residence                    7024
state_code

In [81]:
df['season_holidayed_code']=df['season_holidayed_code'].fillna(0)

In [82]:
df['season_holidayed_code'].value_counts()

2.0    180857
3.0    169635
4.0     99276
1.0     38260
0.0       149
Name: season_holidayed_code, dtype: int64

In [83]:
df['state_code_residence'].value_counts().unique()

array([192377,  59087,  40077,  36820,  30333,  22812,  17352,  13568,
        13468,   9251,   8738,   5644,   5507,   4495,   3346,   2455,
         2446,   2046,   2014,   1583,   1547,   1225,   1074,   1045,
          741,    576,    550,    208,    202,    133,     99,     98,
           86,     72,     31,     29,     18], dtype=int64)

In [84]:
df['state_code_residence'].mode()

0    8.0
dtype: float64

In [85]:
df['state_code_residence']=df['state_code_residence'].fillna(df['state_code_residence'].mode())

In [86]:
##Now we dont have  any null values lets see our data
df.head()

,amount_spent_per_room_night_scaled,booking_date,booking_type_code,channel_code,checkin_date,checkout_date,cluster_code,main_product_code,member_age_buckets,memberid,numberofadults,numberofchildren,persontravellingid,reservation_id,reservationstatusid_code,resort_id,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,Stay_duration,Prior_chekin,Prior_checkout,stay_divide_total,booking_date_y,booking_date_m,booking_date_d,booking_date_w,checkin_date_y,checkin_date_m,checkin_date_d,checkin_date_w,checkout_date_y,checkout_date_m,checkout_date_d,checkout_date_w
0,7.706428,2018-04-05,1,3,2018-04-05,2018-04-06,F,1,F,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,46,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,C,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,3,3,3,1,2.0,7.0,3,3,1.0,0.0,1.0,3.0,2018,4,5,14,2018,4,5,14,2018,4,6,14
1,6.662563,2015-01-23,1,1,2015-04-11,2015-04-16,F,1,F,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,46,03930f033646d073462b35d411616323597715ac4fc398...,A,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,3,3,4,5,2.0,7.0,5,2,5.0,78.0,83.0,0.4,2015,1,23,4,2015,4,11,15,2015,4,16,16
2,7.871602,2015-01-28,1,1,2015-02-01,2015-02-05,E,1,F,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,47,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,A,535fa30d7e25dd8a49f1536779734ec8286108d115da50...,1,5,4,4,2.0,7.0,1,2,4.0,4.0,8.0,0.5,2015,1,28,5,2015,2,1,5,2015,2,5,6
3,5.344943,2015-05-02,1,1,2015-06-11,2015-06-16,D,1,F,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,2,46,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04...,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,2,2,3,5,2.0,7.0,2,2,5.0,40.0,45.0,0.4,2015,5,2,18,2015,6,11,24,2015,6,16,25
4,7.059346,2015-09-02,1,1,2015-12-14,2015-12-19,D,1,F,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,46,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb29...,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,2,2,4,5,2.0,7.0,2,2,5.0,103.0,108.0,0.4,2015,9,2,36,2015,12,14,51,2015,12,19,51


In [87]:
#Creating Family_type
df['family_type'] = df['numberofchildren'].apply(lambda x: 1 if x > 0 else 0)


In [88]:
df['persontravellingid'].unique()

array([  46,   47,   45, 4752, 4753, 4995], dtype=int64)

In [89]:
df['total_pax'].unique()

array([ 3,  2,  1,  4,  6,  5,  7,  8,  9, 10, 12,  0, 11, 20, 14, 21, 13,
       24], dtype=int64)

In [90]:
df['member_age_buckets'].unique()

array(['F', 'H', 'D', 'E', 'I', 'G', 'C', 'B', 'A', 'J'], dtype=object)

In [92]:
df['state_code_residence'].unique()

array([ 7.,  8.,  2.,  5.,  6.,  3.,  4.,  1.,  9., 14., nan, 25., 15.,
       12., 10., 28., 24., 23., 20., 11., 13., 21., 18., 16., 26., 27.,
       22., 19., 31., 34., 32., 30., 33., 29., 37., 35., 36., 38.])

In [94]:
df['state_code_residence'].mode()

0    8.0
dtype: float64

In [95]:
# df['state_code_residence'].isnull().sum()
df['state_code_residence']=df['state_code_residence'].fillna(8)

In [100]:
from sklearn.preprocessing import LabelEncoder
labels=LabelEncoder()
col = ['resort_id','persontravellingid','booking_type_code','channel_code','cluster_code','main_product_code','member_age_buckets','memberid',
       'reservationstatusid_code','resort_region_code','resort_type_code','room_type_booked_code','season_holidayed_code',
      'state_code_residence','state_code_resort','family_type']
for i in col:
    df[i]=labels.fit_transform(df[i])

In [99]:
df['NumberOfPeople']=df['numberofadults']+df['numberofchildren']
df['resort_region_type_code']=df['resort_region_code']+df['resort_type_code']
df['state_code_residence_resort']=df['state_code_residence']+df['state_code_resort']
df['resort_type_cluster_code']=df['resort_type_code']+df['cluster_code']

In [101]:
labels=LabelEncoder()
col = ['resort_type_cluster_code','state_code_residence_resort','resort_region_type_code']
for i in col:
    df[i]=labels.fit_transform(df[i])

In [102]:
df.head()

,amount_spent_per_room_night_scaled,booking_date,booking_type_code,channel_code,checkin_date,checkout_date,cluster_code,main_product_code,member_age_buckets,memberid,numberofadults,numberofchildren,persontravellingid,reservation_id,reservationstatusid_code,resort_id,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,Stay_duration,Prior_chekin,Prior_checkout,stay_divide_total,booking_date_y,booking_date_m,booking_date_d,booking_date_w,checkin_date_y,checkin_date_m,checkin_date_d,checkin_date_w,checkout_date_y,checkout_date_m,checkout_date_d,checkout_date_w,family_type,NumberOfPeople,resort_region_type_code,state_code_residence_resort,resort_type_cluster_code
0,7.706428,2018-04-05,0,2,2018-04-05,2018-04-06,5,0,5,34521,2,0,1,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,2,7,2,3,2,1,2,6,2,3,1.0,0.0,1.0,3.0,2018,4,5,14,2018,4,5,14,2018,4,6,14,0,2,3,8,6
1,6.662563,2015-01-23,0,0,2015-04-11,2015-04-16,5,0,5,34521,2,0,1,03930f033646d073462b35d411616323597715ac4fc398...,0,1,2,3,3,5,2,6,4,2,5.0,78.0,83.0,0.4,2015,1,23,4,2015,4,11,15,2015,4,16,16,0,2,3,10,6
2,7.871602,2015-01-28,0,0,2015-02-01,2015-02-05,4,0,5,34521,2,0,2,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,0,9,0,5,3,4,2,6,0,2,4.0,4.0,8.0,0.5,2015,1,28,5,2015,2,1,5,2015,2,5,6,0,2,3,6,7
3,5.344943,2015-05-02,0,0,2015-06-11,2015-06-16,3,0,5,34521,2,2,1,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04...,0,25,1,2,2,5,2,6,1,2,5.0,40.0,45.0,0.4,2015,5,2,18,2015,6,11,24,2015,6,16,25,1,4,2,7,3
4,7.059346,2015-09-02,0,0,2015-12-14,2015-12-19,3,0,5,34521,2,0,1,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb29...,0,25,1,2,3,5,2,6,1,2,5.0,103.0,108.0,0.4,2015,9,2,36,2015,12,14,51,2015,12,19,51,0,2,2,7,3


In [126]:
# df[['family_type', 'persontravellingid', 'member_age_buckets', 'cluster_code','reservationstatusid_code','booking_type_code',
#         'state_code_resort','state_code_residence','season_holidayed_code','room_type_booked_code','resort_type_code',
#        'resort_region_code','main_product_code','channel_code','NumberOfPeople','resort_region_type_code','state_code_residence_resort','resort_type_cluster_code','member_age_buckets']] = df[['family_type', 'persontravellingid', 'member_age_buckets', 'cluster_code','reservationstatusid_code','booking_type_code',
#         'state_code_resort','state_code_residence','season_holidayed_code','room_type_booked_code','resort_type_code',
#        'resort_region_code','main_product_code','channel_code','NumberOfPeople','resort_region_type_code','state_code_residence_resort','resort_type_cluster_code','member_age_buckets']].astype('category')


In [103]:
df.drop(['booking_date','checkin_date','checkout_date'],axis=1,inplace=True)


In [104]:
df.shape

(488177, 42)

In [105]:
df.drop(['reservation_id'],axis=1,inplace=True)


In [106]:
df.shape

(488177, 41)

In [107]:
df.head()

,amount_spent_per_room_night_scaled,booking_type_code,channel_code,cluster_code,main_product_code,member_age_buckets,memberid,numberofadults,numberofchildren,persontravellingid,reservationstatusid_code,resort_id,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,Stay_duration,Prior_chekin,Prior_checkout,stay_divide_total,booking_date_y,booking_date_m,booking_date_d,booking_date_w,checkin_date_y,checkin_date_m,checkin_date_d,checkin_date_w,checkout_date_y,checkout_date_m,checkout_date_d,checkout_date_w,family_type,NumberOfPeople,resort_region_type_code,state_code_residence_resort,resort_type_cluster_code
0,7.706428,0,2,5,0,5,34521,2,0,1,2,7,2,3,2,1,2,6,2,3,1.0,0.0,1.0,3.0,2018,4,5,14,2018,4,5,14,2018,4,6,14,0,2,3,8,6
1,6.662563,0,0,5,0,5,34521,2,0,1,0,1,2,3,3,5,2,6,4,2,5.0,78.0,83.0,0.4,2015,1,23,4,2015,4,11,15,2015,4,16,16,0,2,3,10,6
2,7.871602,0,0,4,0,5,34521,2,0,2,0,9,0,5,3,4,2,6,0,2,4.0,4.0,8.0,0.5,2015,1,28,5,2015,2,1,5,2015,2,5,6,0,2,3,6,7
3,5.344943,0,0,3,0,5,34521,2,2,1,0,25,1,2,2,5,2,6,1,2,5.0,40.0,45.0,0.4,2015,5,2,18,2015,6,11,24,2015,6,16,25,1,4,2,7,3
4,7.059346,0,0,3,0,5,34521,2,0,1,0,25,1,2,3,5,2,6,1,2,5.0,103.0,108.0,0.4,2015,9,2,36,2015,12,14,51,2015,12,19,51,0,2,2,7,3


#### memberid and resort_id both are categorical variable hence we use label encoder

In [108]:
df.dtypes

amount_spent_per_room_night_scaled    float64
booking_type_code                       int64
channel_code                            int64
cluster_code                            int64
main_product_code                       int64
member_age_buckets                      int64
memberid                                int64
numberofadults                          int64
numberofchildren                        int64
persontravellingid                      int64
reservationstatusid_code                int64
resort_id                               int64
resort_region_code                      int64
resort_type_code                        int64
room_type_booked_code                   int64
roomnights                              int64
season_holidayed_code                   int64
state_code_residence                    int64
state_code_resort                       int64
total_pax                               int64
Stay_duration                         float64
Prior_chekin                      

In [109]:
df['persontravellingid'].head()

0    1
1    1
2    2
3    1
4    1
Name: persontravellingid, dtype: int64

In [110]:
df['resort_type_cluster_code'].isnull().sum()

0

In [111]:
df.dtypes

amount_spent_per_room_night_scaled    float64
booking_type_code                       int64
channel_code                            int64
cluster_code                            int64
main_product_code                       int64
member_age_buckets                      int64
memberid                                int64
numberofadults                          int64
numberofchildren                        int64
persontravellingid                      int64
reservationstatusid_code                int64
resort_id                               int64
resort_region_code                      int64
resort_type_code                        int64
room_type_booked_code                   int64
roomnights                              int64
season_holidayed_code                   int64
state_code_residence                    int64
state_code_resort                       int64
total_pax                               int64
Stay_duration                         float64
Prior_chekin                      

In [112]:
df.head()

,amount_spent_per_room_night_scaled,booking_type_code,channel_code,cluster_code,main_product_code,member_age_buckets,memberid,numberofadults,numberofchildren,persontravellingid,reservationstatusid_code,resort_id,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,Stay_duration,Prior_chekin,Prior_checkout,stay_divide_total,booking_date_y,booking_date_m,booking_date_d,booking_date_w,checkin_date_y,checkin_date_m,checkin_date_d,checkin_date_w,checkout_date_y,checkout_date_m,checkout_date_d,checkout_date_w,family_type,NumberOfPeople,resort_region_type_code,state_code_residence_resort,resort_type_cluster_code
0,7.706428,0,2,5,0,5,34521,2,0,1,2,7,2,3,2,1,2,6,2,3,1.0,0.0,1.0,3.0,2018,4,5,14,2018,4,5,14,2018,4,6,14,0,2,3,8,6
1,6.662563,0,0,5,0,5,34521,2,0,1,0,1,2,3,3,5,2,6,4,2,5.0,78.0,83.0,0.4,2015,1,23,4,2015,4,11,15,2015,4,16,16,0,2,3,10,6
2,7.871602,0,0,4,0,5,34521,2,0,2,0,9,0,5,3,4,2,6,0,2,4.0,4.0,8.0,0.5,2015,1,28,5,2015,2,1,5,2015,2,5,6,0,2,3,6,7
3,5.344943,0,0,3,0,5,34521,2,2,1,0,25,1,2,2,5,2,6,1,2,5.0,40.0,45.0,0.4,2015,5,2,18,2015,6,11,24,2015,6,16,25,1,4,2,7,3
4,7.059346,0,0,3,0,5,34521,2,0,1,0,25,1,2,3,5,2,6,1,2,5.0,103.0,108.0,0.4,2015,9,2,36,2015,12,14,51,2015,12,19,51,0,2,2,7,3


In [95]:
# df.drop(['memberid','resort_id'],axis=1,inplace=True)


In [96]:
df.head()

,amount_spent_per_room_night_scaled,booking_type_code,channel_code,cluster_code,main_product_code,member_age_buckets,memberid,numberofadults,numberofchildren,persontravellingid,reservationstatusid_code,resort_id,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,Stay_duration,Prior_chekin,Prior_checkout,stay_divide_total,booking_date_y,booking_date_m,booking_date_d,booking_date_w,checkin_date_y,checkin_date_m,checkin_date_d,checkin_date_w,checkout_date_y,checkout_date_m,checkout_date_d,checkout_date_w,family_type,NumberOfPeople,resort_region_type_code,state_code_residence_resort,resort_type_cluster_code
0,7.706428,0,2,5,0,5,34521,2,0,1,2,7,2,3,2,1,2,7,2,3,1.0,0.0,1.0,3.0,2018,4,5,14,2018,4,5,14,2018,4,6,14,0,2,3,9,6
1,6.662563,0,0,5,0,5,34521,2,0,1,0,1,2,3,3,5,2,7,4,2,5.0,78.0,83.0,0.4,2015,1,23,4,2015,4,11,15,2015,4,16,16,0,2,3,11,6
2,7.871602,0,0,4,0,5,34521,2,0,2,0,9,0,5,3,4,2,7,0,2,4.0,4.0,8.0,0.5,2015,1,28,5,2015,2,1,5,2015,2,5,6,0,2,3,7,7
3,5.344943,0,0,3,0,5,34521,2,2,1,0,25,1,2,2,5,2,7,1,2,5.0,40.0,45.0,0.4,2015,5,2,18,2015,6,11,24,2015,6,16,25,1,4,2,8,3
4,7.059346,0,0,3,0,5,34521,2,0,1,0,25,1,2,3,5,2,7,1,2,5.0,103.0,108.0,0.4,2015,9,2,36,2015,12,14,51,2015,12,19,51,0,2,2,8,3


In [113]:
df.shape

(488177, 41)

In [114]:
df.dtypes

amount_spent_per_room_night_scaled    float64
booking_type_code                       int64
channel_code                            int64
cluster_code                            int64
main_product_code                       int64
member_age_buckets                      int64
memberid                                int64
numberofadults                          int64
numberofchildren                        int64
persontravellingid                      int64
reservationstatusid_code                int64
resort_id                               int64
resort_region_code                      int64
resort_type_code                        int64
room_type_booked_code                   int64
roomnights                              int64
season_holidayed_code                   int64
state_code_residence                    int64
state_code_resort                       int64
total_pax                               int64
Stay_duration                         float64
Prior_chekin                      

In [115]:
df.columns

Index(['amount_spent_per_room_night_scaled', 'booking_type_code',
       'channel_code', 'cluster_code', 'main_product_code',
       'member_age_buckets', 'memberid', 'numberofadults', 'numberofchildren',
       'persontravellingid', 'reservationstatusid_code', 'resort_id',
       'resort_region_code', 'resort_type_code', 'room_type_booked_code',
       'roomnights', 'season_holidayed_code', 'state_code_residence',
       'state_code_resort', 'total_pax', 'Stay_duration', 'Prior_chekin',
       'Prior_checkout', 'stay_divide_total', 'booking_date_y',
       'booking_date_m', 'booking_date_d', 'booking_date_w', 'checkin_date_y',
       'checkin_date_m', 'checkin_date_d', 'checkin_date_w', 'checkout_date_y',
       'checkout_date_m', 'checkout_date_d', 'checkout_date_w', 'family_type',
       'NumberOfPeople', 'resort_region_type_code',
       'state_code_residence_resort', 'resort_type_cluster_code'],
      dtype='object')

In [116]:
df['member_age_buckets'].unique()

array([5, 7, 3, 4, 8, 6, 2, 1, 0, 9], dtype=int64)

In [117]:
df['member_age_buckets'].value_counts()

3    95950
4    95640
5    73197
2    64775
7    54662
6    51999
1    27557
8    12054
0    10316
9     2027
Name: member_age_buckets, dtype: int64

In [105]:
# booking_type_code= pd.get_dummies(df['booking_type_code'],prefix='booking_type_code')
# channel_code= pd.get_dummies(df['channel_code'],prefix='channel_code')
# cluster_code= pd.get_dummies(df['cluster_code'],prefix='cluster_code')
# main_product_code= pd.get_dummies(df['main_product_code'],prefix='main_product_code')
# member_age_buckets= pd.get_dummies(df['member_age_buckets'],prefix='member_age_buckets')
# #memberid= pd.get_dummies(df['memberid'],prefix='memberid')
# reservationstatusid_code= pd.get_dummies(df['reservationstatusid_code'],prefix='reservationstatusid_code')
# resort_region_code= pd.get_dummies(df['resort_region_code'],prefix='resort_region_code')
# resort_type_code= pd.get_dummies(df['resort_type_code'],prefix='resort_type_code')
# room_type_booked_code= pd.get_dummies(df['room_type_booked_code'],prefix='room_type_booked_code')
# season_holidayed_code= pd.get_dummies(df['season_holidayed_code'],prefix='season_holidayed_code')
# family_type= pd.get_dummies(df['family_type'],prefix='family_type')
# resort_region_type_code= pd.get_dummies(df['resort_region_type_code'],prefix='resort_region_type_code')
# state_code_residence_resort= pd.get_dummies(df['state_code_residence_resort'],prefix='state_code_residence_resort')
# resort_type_cluster_code= pd.get_dummies(df['resort_type_cluster_code'],prefix='resort_type_cluster_code')
# persontravellingid=pd.get_dummies(df['persontravellingid'],prefix='persontravellingid')

In [106]:
# df=pd.concat([df,booking_type_code],axis=1)
# df=pd.concat([df,channel_code],axis=1)
# df=pd.concat([df,cluster_code],axis=1)
# df=pd.concat([df,main_product_code],axis=1)
# df=pd.concat([df,member_age_buckets],axis=1)
# #df=pd.concat([df,memberid],axis=1)
# df=pd.concat([df,reservationstatusid_code],axis=1)
# df=pd.concat([df,resort_region_code],axis=1)
# df=pd.concat([df,resort_type_code],axis=1)
# df=pd.concat([df,room_type_booked_code],axis=1)
# df=pd.concat([df,season_holidayed_code],axis=1)
# df=pd.concat([df,family_type],axis=1)
# df=pd.concat([df,resort_region_type_code],axis=1)
# df=pd.concat([df,state_code_residence_resort],axis=1)
# df=pd.concat([df,resort_type_cluster_code],axis=1)
# df=pd.concat([df,persontravellingid],axis=1)

In [120]:
df.shape

(488177, 41)

In [121]:
import statsmodels.api as sm

### Train and Test splitting

In [165]:
dftrain=df[df['amount_spent_per_room_night_scaled'].isnull()!=True]
dftest=df[df['amount_spent_per_room_night_scaled'].isnull()==True]

In [166]:
X,y=dftrain.drop('amount_spent_per_room_night_scaled',axis=1),dftrain['amount_spent_per_room_night_scaled']
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.3,random_state = 1994)

In [167]:
dftrain.head()

,amount_spent_per_room_night_scaled,booking_type_code,channel_code,cluster_code,main_product_code,member_age_buckets,memberid,numberofadults,numberofchildren,persontravellingid,reservationstatusid_code,resort_id,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,Stay_duration,Prior_chekin,Prior_checkout,stay_divide_total,booking_date_y,booking_date_m,booking_date_d,booking_date_w,checkin_date_y,checkin_date_m,checkin_date_d,checkin_date_w,checkout_date_y,checkout_date_m,checkout_date_d,checkout_date_w,family_type,NumberOfPeople,resort_region_type_code,state_code_residence_resort,resort_type_cluster_code
0,7.706428,0,2,5,0,5,34521,2,0,1,2,7,2,3,2,1,2,6,2,3,1.0,0.0,1.0,3.0,2018,4,5,14,2018,4,5,14,2018,4,6,14,0,2,3,8,6
1,6.662563,0,0,5,0,5,34521,2,0,1,0,1,2,3,3,5,2,6,4,2,5.0,78.0,83.0,0.4,2015,1,23,4,2015,4,11,15,2015,4,16,16,0,2,3,10,6
2,7.871602,0,0,4,0,5,34521,2,0,2,0,9,0,5,3,4,2,6,0,2,4.0,4.0,8.0,0.5,2015,1,28,5,2015,2,1,5,2015,2,5,6,0,2,3,6,7
3,5.344943,0,0,3,0,5,34521,2,2,1,0,25,1,2,2,5,2,6,1,2,5.0,40.0,45.0,0.4,2015,5,2,18,2015,6,11,24,2015,6,16,25,1,4,2,7,3
4,7.059346,0,0,3,0,5,34521,2,0,1,0,25,1,2,3,5,2,6,1,2,5.0,103.0,108.0,0.4,2015,9,2,36,2015,12,14,51,2015,12,19,51,0,2,2,7,3


In [124]:
def stepwise_selection(X_train, y_train, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))
                
        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

result = stepwise_selection(X_train, y_train)

print('resulting features:')

D:\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.


Add  booking_date_y                 with p-value 0.0
Add  resort_region_code             with p-value 0.0
Add  channel_code                   with p-value 0.0
Add  roomnights                     with p-value 0.0
Add  family_type                    with p-value 0.0
Add  Stay_duration                  with p-value 0.0
Add  season_holidayed_code          with p-value 0.0
Add  NumberOfPeople                 with p-value 0.0
Add  persontravellingid             with p-value 0.0
Add  stay_divide_total              with p-value 0.0
Add  main_product_code              with p-value 0.0
Add  resort_id                      with p-value 1.53347e-204
Add  checkin_date_w                 with p-value 8.77953e-43
Add  checkout_date_m                with p-value 4.96714e-56
Add  room_type_booked_code          with p-value 1.69804e-37
Add  resort_region_type_code        with p-value 2.28857e-50
Add  resort_type_code               with p-value 4.38522e-102
Add  state_code_resort              with p-value 

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.


Drop NumberOfPeople                 with p-value 0.458982
Add  total_pax                      with p-value 1.43222e-22
Add  cluster_code                   with p-value 1.98398e-20
Add  resort_type_cluster_code       with p-value 2.40075e-101
Add  state_code_residence_resort    with p-value 2.5611e-17
Add  state_code_residence           with p-value 7.9835e-32
Add  booking_type_code              with p-value 4.30888e-10
resulting features:


In [125]:
result

['booking_date_y',
 'resort_region_code',
 'channel_code',
 'roomnights',
 'family_type',
 'Stay_duration',
 'season_holidayed_code',
 'persontravellingid',
 'stay_divide_total',
 'main_product_code',
 'resort_id',
 'checkin_date_w',
 'checkout_date_m',
 'room_type_booked_code',
 'resort_region_type_code',
 'resort_type_code',
 'state_code_resort',
 'checkin_date_m',
 'numberofchildren',
 'numberofadults',
 'total_pax',
 'cluster_code',
 'resort_type_cluster_code',
 'state_code_residence_resort',
 'state_code_residence',
 'booking_type_code']

In [141]:
X_train=X_train[['booking_date_y',
 'resort_region_code',
 'channel_code',
 'roomnights',
 'family_type',
 'Stay_duration',
 'season_holidayed_code',
 'persontravellingid',
 'stay_divide_total',
 'main_product_code',
 'resort_id',
 'checkin_date_w',
 'checkout_date_m',
 'room_type_booked_code',
 'resort_region_type_code',
 'resort_type_code',
 'state_code_resort',
 'checkin_date_m',
 'numberofchildren',
 'numberofadults',
 'total_pax',
 'cluster_code',
 'resort_type_cluster_code',
 'state_code_residence_resort',
 'state_code_residence',
 'booking_type_code']]

In [142]:
X_train.shape

(238988, 26)

In [143]:
# categorical_features_indices = np.where(X_train.dtypes =='category')[0]
# categorical_features_indices

### Hyper parameter Learning Rate

## CV regressor

In [208]:
CatBoostRegressor?

In [210]:
categorical_features_indices = np.where(X_train.dtypes =='category')[0]
categorical_features_indices

array([ 0,  1,  2,  3,  8,  9, 11, 12, 13, 15, 16, 17, 35, 36, 37, 38, 39],
      dtype=int64)

In [209]:
X_train[['family_type', 'persontravellingid', 'member_age_buckets', 'cluster_code','reservationstatusid_code','booking_type_code',
        'state_code_resort','state_code_residence','season_holidayed_code','room_type_booked_code','resort_type_code',
       'resort_region_code','main_product_code','channel_code','NumberOfPeople','resort_region_type_code','state_code_residence_resort','resort_type_cluster_code','member_age_buckets']] = X_train[['family_type', 'persontravellingid', 'member_age_buckets', 'cluster_code','reservationstatusid_code','booking_type_code',
        'state_code_resort','state_code_residence','season_holidayed_code','room_type_booked_code','resort_type_code',
       'resort_region_code','main_product_code','channel_code','NumberOfPeople','resort_region_type_code','state_code_residence_resort','resort_type_cluster_code','member_age_buckets']].astype('category')

D:\Anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [ ]:
(iterations=260,
                             learning_rate=0.9,
                             depth=2,
                             eval_metric='RMSE',
                             random_seed = 23,
                             loss_function='RMSE',
                             bagging_temperature =0.2,
                             od_type='Iter',
                             cat_features=[0,  1,  2,  3,  8,  9, 11, 12, 13, 15, 16, 17, 35, 36, 37, 38, 39],
                             metric_period = 75,
                             od_wait=100)

In [216]:
from catboost import CatBoostRegressor, CatBoostClassifier
cat = CatBoostRegressor(iterations = 10000,
                                #task_type = 'GPU',
                                eval_metric='RMSE',
                                loss_function= 'RMSE',
                                cat_features=[0,  1,  2,  3,  8,  9, 11, 12, 13, 15, 16, 17, 35, 36, 37, 38, 39],
                                #learning_rate=0.003
                                #scale_pos_weight = 1.55,
                                #depth=7
                                )

cat.fit(X_train, y_train)

KeyError: 3

In [144]:
from tqdm import tqdm


In [145]:
from mlxtend.regressor import StackingRegressor
from mlxtend.data import boston_housing_data
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import numpy as np

In [146]:
from sklearn.base import TransformerMixin
from sklearn.datasets import make_regression
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge

In [147]:
# def Snippet_199(): 
#     print()
#     print(format('How to find optimal parameters for CatBoost using GridSearchCV for Regression','*^82'))    
    
#     import warnings
#     warnings.filterwarnings("ignore")
    
#     # load libraries
#     from sklearn import datasets
#     from sklearn.model_selection import train_test_split
#     from sklearn.model_selection import GridSearchCV
#     from catboost import CatBoostRegressor

#     # load the iris datasets
# #     dataset = datasets.load_boston()
# #     X = dataset.data; y = dataset.target
# #     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

#     model = CatBoostRegressor()
#     parameters = {'depth'         : [2],
#                   'learning_rate' : [0.8,0.9,0.95,1],
#                   'iterations'    : [100,200]
#                  }
#     grid = GridSearchCV(estimator=model, param_grid = parameters, cv = 2, n_jobs=-1)
#     grid.fit(X_train, y_train)    

#     # Results from Grid Search
#     print("\n========================================================")
#     print(" Results from Grid Search " )
#     print("========================================================")    
    
#     print("\n The best estimator across ALL searched params:\n",
#           grid.best_estimator_)
    
#     print("\n The best score across ALL searched params:\n",
#           grid.best_score_)
    
#     print("\n The best parameters across ALL searched params:\n",
#           grid.best_params_)
    
#     print("\n ========================================================")
# Snippet_199()

In [148]:
# !pip install scikit-optimize


In [149]:
X_train.head()

,booking_date_y,resort_region_code,channel_code,roomnights,family_type,Stay_duration,season_holidayed_code,persontravellingid,stay_divide_total,main_product_code,resort_id,checkin_date_w,checkout_date_m,room_type_booked_code,resort_region_type_code,resort_type_code,state_code_resort,checkin_date_m,numberofchildren,numberofadults,total_pax,cluster_code,resort_type_cluster_code,state_code_residence_resort,state_code_residence,booking_type_code
167971,2016,0,2,6,0,3.0,2,0,0.333333,1,13,17,4,3,0,1,0,4,0,3,1,4,3,5,5,0
132271,2015,2,2,2,0,2.0,2,0,2.500000,1,20,25,6,3,3,3,2,6,0,2,5,0,2,6,4,0
182109,2017,2,0,4,1,2.0,2,0,1.000000,1,8,17,4,3,4,4,4,4,1,4,2,5,7,5,1,0
56066,2015,0,0,3,1,2.0,4,0,2.000000,1,22,6,2,1,0,1,5,2,1,3,4,5,4,6,1,0
333069,2018,0,0,2,1,2.0,3,1,1.000000,2,24,25,6,3,3,5,0,6,1,1,2,4,7,7,7,0


In [150]:
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from skopt.plots import plot_convergence
import time

In [172]:
X.shape

(341412, 40)

In [181]:
from sklearn.model_selection import KFold

K = 5
kf = KFold(n_splits = K, random_state = 3228, shuffle = True)

In [ ]:
# preds=[]
# for i in range(len(xgb_preds[0])):
#     sum=0
#     for j in range(K):
#         sum+=xgb_preds[j][i]
#     preds.append(sum / K)

# output = pd.DataFrame({'id': id_test, 'target': preds})
# output.to_csv("{}-foldCV_avg_sub.csv".format(K), index=False) 

In [195]:
from sklearn.metrics import mean_squared_error
from sklearn import cross_validation, linear_model



ImportError: cannot import name 'cross_validation' from 'sklearn' (D:\Anaconda3\lib\site-packages\sklearn\__init__.py)

In [207]:
from sklearn.metrics import mean_squared_error
mean_squared_error?
cross_val_score?

In [205]:
scores = cross_val_score(cb_model, X_train, y_train, scoring=(estimator, X, y), cv=5)


ValueError: scoring value <function mean_squared_error at 0x0000025B9C3A9158> looks like it is a metric function rather than a scorer. A scorer should require an estimator as its first parameter. Please use `make_scorer` to convert a metric to a scorer.

In [191]:
from sklearn.model_selection import cross_val_score  
all_accuracies = cross_val_score(estimator=cb_model, X=X_train, y=y_train, cv=5)  

0:	learn: 1.3364622	total: 20.4ms	remaining: 5.29s
75:	learn: 0.9816462	total: 1.48s	remaining: 3.58s
150:	learn: 0.9777455	total: 2.93s	remaining: 2.12s
225:	learn: 0.9754558	total: 4.37s	remaining: 657ms
259:	learn: 0.9744619	total: 5.02s	remaining: 0us
0:	learn: 1.3361243	total: 20.9ms	remaining: 5.42s
75:	learn: 0.9826097	total: 1.49s	remaining: 3.61s
150:	learn: 0.9786471	total: 2.94s	remaining: 2.12s
225:	learn: 0.9758527	total: 4.39s	remaining: 661ms
259:	learn: 0.9748862	total: 5.05s	remaining: 0us
0:	learn: 1.3363956	total: 21ms	remaining: 5.44s
75:	learn: 0.9828281	total: 1.55s	remaining: 3.75s
150:	learn: 0.9784574	total: 3.12s	remaining: 2.25s
225:	learn: 0.9759982	total: 4.62s	remaining: 696ms
259:	learn: 0.9750894	total: 5.29s	remaining: 0us
0:	learn: 1.3379380	total: 22.5ms	remaining: 5.82s
75:	learn: 0.9856359	total: 1.67s	remaining: 4.03s
150:	learn: 0.9804667	total: 3.26s	remaining: 2.36s
225:	learn: 0.9779926	total: 4.79s	remaining: 721ms
259:	learn: 0.9771964	total:

In [192]:
print(all_accuracies)  


[0.98540949 0.98404124 0.9824672  0.9740169  0.99287939]


In [233]:
y.shape

(341412,)

In [232]:
X[['family_type', 'persontravellingid', 'member_age_buckets', 'cluster_code','reservationstatusid_code','booking_type_code',
        'state_code_resort','state_code_residence','season_holidayed_code','room_type_booked_code','resort_type_code',
       'resort_region_code','main_product_code','channel_code','NumberOfPeople','resort_region_type_code','state_code_residence_resort','resort_type_cluster_code','member_age_buckets']] = X[['family_type', 'persontravellingid', 'member_age_buckets', 'cluster_code','reservationstatusid_code','booking_type_code',
        'state_code_resort','state_code_residence','season_holidayed_code','room_type_booked_code','resort_type_code',
       'resort_region_code','main_product_code','channel_code','NumberOfPeople','resort_region_type_code','state_code_residence_resort','resort_type_cluster_code','member_age_buckets']].astype('category')

In [224]:
dftest[['family_type', 'persontravellingid', 'member_age_buckets', 'cluster_code','reservationstatusid_code','booking_type_code',
        'state_code_resort','state_code_residence','season_holidayed_code','room_type_booked_code','resort_type_code',
       'resort_region_code','main_product_code','channel_code','NumberOfPeople','resort_region_type_code','state_code_residence_resort','resort_type_cluster_code','member_age_buckets']] = dftest[['family_type', 'persontravellingid', 'member_age_buckets', 'cluster_code','reservationstatusid_code','booking_type_code',
        'state_code_resort','state_code_residence','season_holidayed_code','room_type_booked_code','resort_type_code',
       'resort_region_code','main_product_code','channel_code','NumberOfPeople','resort_region_type_code','state_code_residence_resort','resort_type_cluster_code','member_age_buckets']].astype('category')

In [225]:
dftrain[['family_type', 'persontravellingid', 'member_age_buckets', 'cluster_code','reservationstatusid_code','booking_type_code',
        'state_code_resort','state_code_residence','season_holidayed_code','room_type_booked_code','resort_type_code',
       'resort_region_code','main_product_code','channel_code','NumberOfPeople','resort_region_type_code','state_code_residence_resort','resort_type_cluster_code','member_age_buckets']] = dftrain[['family_type', 'persontravellingid', 'member_age_buckets', 'cluster_code','reservationstatusid_code','booking_type_code',
        'state_code_resort','state_code_residence','season_holidayed_code','room_type_booked_code','resort_type_code',
       'resort_region_code','main_product_code','channel_code','NumberOfPeople','resort_region_type_code','state_code_residence_resort','resort_type_cluster_code','member_age_buckets']].astype('category')

In [235]:
categorical_features_indices = np.where(X.dtypes =='category')[0]
categorical_features_indices

array([ 0,  1,  2,  3,  8,  9, 11, 12, 13, 15, 16, 17, 35, 36, 37, 38, 39],
      dtype=int64)

In [230]:
dftrain.shape

(341412, 41)

In [231]:
X.shape

(341412, 40)

In [237]:
cb_model =CatBoostRegressor(#iterations=260,
                             learning_rate=0.9,
                             depth=2,
                             eval_metric='RMSE',
                             random_seed = 23,
                             loss_function='RMSE',
                             bagging_temperature =0.2,
                             od_type='Iter',
                             metric_period = 75,
                             od_wait=100)
cb_model.fit(X, y)
# Get predictions
#preds = cb_model.predict(X_val)

0:	learn: 1.3358937	total: 31ms	remaining: 30.9s
75:	learn: 0.9844088	total: 2.4s	remaining: 29.1s
150:	learn: 0.9800165	total: 4.74s	remaining: 26.6s
225:	learn: 0.9779232	total: 7.15s	remaining: 24.5s
300:	learn: 0.9763208	total: 9.54s	remaining: 22.1s
375:	learn: 0.9752238	total: 11.9s	remaining: 19.7s
450:	learn: 0.9741325	total: 14.3s	remaining: 17.5s
525:	learn: 0.9732623	total: 16.8s	remaining: 15.1s
600:	learn: 0.9724603	total: 19.2s	remaining: 12.7s
675:	learn: 0.9717632	total: 21.5s	remaining: 10.3s
750:	learn: 0.9711409	total: 23.9s	remaining: 7.93s
825:	learn: 0.9705609	total: 26.3s	remaining: 5.53s
900:	learn: 0.9698715	total: 28.6s	remaining: 3.14s
975:	learn: 0.9692690	total: 31.2s	remaining: 767ms
999:	learn: 0.9690843	total: 32.1s	remaining: 0us


In [ ]:
dftest

In [238]:
cb_model.predict(dftest)

array([8.11719261, 7.56345223, 7.41181551, ..., 8.51254805, 8.96754324,
       7.11684781])

In [175]:
dftest.shape

(146765, 41)

In [176]:
X_train.shape

(238988, 40)

In [177]:
cb_model.predict(dftest)

array([8.09604729, 7.42942987, 7.49248347, ..., 8.40688891, 8.78374616,
       7.253836  ])

In [239]:
sample_submission=pd.read_csv('sample_submission.csv')

In [240]:
s=pd.DataFrame({'reservation_id':sample_submission['reservation_id'],'amount_spent_per_room_night_scaled':cb_model.predict(dftest)})
s.head()


s.to_csv('Submission1.csv',index=False)

In [309]:
model = CatBoostRegressor(iterations=100,
                          learning_rate=0.92,
                          depth=2)

In [218]:
from sklearn.feature_selection import RFE
#create the RFE model and select 3 attributes
rfcr = CatBoostRegressor(iterations=1000,
                             learning_rate=0.9,
                             depth=2,
                         
                             eval_metric='RMSE',
                             random_seed = 23,
                             loss_function='RMSE',
                             bagging_temperature =0.2,
                             od_type='Iter',
                             metric_period = 75,
                             od_wait=100)
clf_LR = rfcr
clf_LR.fit(X_train,y_train)
rfe = RFE(clf_LR, 10)
rfe = rfe.fit(X_train,y_train)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

CatBoostError: c:/goagent/pipelines/buildmaster/catboost.git/catboost/cuda/cuda_lib/cuda_base.h:281: CUDA error 35: CUDA driver version is insufficient for CUDA runtime version

In [139]:
CatBoostRegressor?

In [323]:
cb_model =CatBoostRegressor(iterations=252,
                             learning_rate=0.9,
                             depth=2,
                             eval_metric='RMSE',
                             random_seed = 23,
                             loss_function='RMSE',
                             bagging_temperature =0.2,
                             od_type='Iter',
                             metric_period = 75,
                             od_wait=100)
cb_model.fit(X_train, y_train)
# Get predictions
preds = cb_model.predict(X_val)


0:	learn: 1.3351515	total: 28.9ms	remaining: 7.26s
75:	learn: 0.9844446	total: 1.97s	remaining: 4.55s
150:	learn: 0.9806787	total: 3.58s	remaining: 2.39s
225:	learn: 0.9785334	total: 5.18s	remaining: 596ms
251:	learn: 0.9780201	total: 5.75s	remaining: 0us


In [324]:
dftest.shape

(146765, 101)

In [325]:
dftest=dftest[['cluster_code_1',
 'booking_date_y',
 'season_holidayed_code_2',
 'room_type_booked_code_6',
 'room_type_booked_code_8',
 'Stay_duration',
 'roomnights',
 'reservationstatusid_code',
 'cluster_code_2',
 'state_code_residence',
 'main_product_code_2',
 'state_code_resort',
 'total_pax',
 'resort_id',
 'resort_type_code_0',
 'state_code_residence_resort_2',
 'family_type_2',
 'room_type_booked_code_2',
 'reservationstatusid_code_2',
 'resort_region_code_6',
 'resort_region_code_7',
 'booking_type_code_0',
 'resort_region_code_4',
 'resort_region_type_code_1',
 'resort_region_type_code_0',
 'memberid',
 'resort_region_type_code',
 'memberid_1',
 'memberid_0',
 'checkin_date_m',
 'checkout_date_m',
 'room_type_booked_code_22',
 'resort_region_code_13',
 'resort_region_code_20',
 'main_product_code',
 'resort_region_code_2',
 'resort_region_code_11',
 'room_type_booked_code_20',
 'resort_region_code_1',
 'room_type_booked_code_7',
 'room_type_booked_code_5',
 'room_type_booked_code_1',
 'room_type_booked_code_30',
 'family_type_0',
 'state_code_residence_resort_0',
 'cluster_code',
 'resort_type_code_4',
 'booking_type_code_4',
 'reservationstatusid_code_3',
 'room_type_booked_code',
 'room_type_booked_code_3',
 'room_type_booked_code_4',
 'booking_type_code_5',
 'main_product_code_1',
 'resort_type_code_3',
 'main_product_code_5',
 'booking_type_code_3',
 'resort_region_code_16',
 'resort_region_code_35',
 'resort_type_cluster_code_3',
 'persontravellingid_3',
 'channel_code_3',
 'resort_region_code_5',
 'resort_type_cluster_code_8',
 'channel_code_8',
 'persontravellingid_8',
 'resort_region_code_0',
 'numberofadults',
 'resort_region_code_9',
 'resort_region_code_17',
 'booking_date_m',
 'channel_code',
 'persontravellingid',
 'resort_type_cluster_code',
 'numberofchildren',
 'room_type_booked_code_33',
 'resort_region_code_38',
 'persontravellingid_4',
 'state_code_residence_resort_1',
 'channel_code_4',
 'family_type_1',
 'resort_type_cluster_code_4',
 'season_holidayed_code',
 'season_holidayed_code_0',
 'season_holidayed_code_1',
 'room_type_booked_code_0',
 'resort_region_code_3',
 'resort_region_code_30',
 'resort_region_code_10',
 'room_type_booked_code_31',
 'resort_region_code_28',
 'state_code_residence_resort_5',
 'family_type_5',
 'persontravellingid_0',
 'persontravellingid_9',
 'channel_code_0',
 'resort_type_cluster_code_0',
 'resort_type_cluster_code_9',
 'channel_code_9',
 'room_type_booked_code_25',
 'resort_region_code_19']]

In [326]:
dftest.shape

(146765, 101)

In [327]:
cb_model.predict(dftest)


array([7.9047469 , 7.37856509, 7.51383464, ..., 8.34739619, 7.96104928,
       7.22232088])

In [303]:
dftest.iloc[:,1:].head()

,booking_type_code,channel_code,cluster_code,main_product_code,member_age_buckets,numberofadults,numberofchildren,persontravellingid,reservationstatusid_code,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,Stay_duration,booking_date_y,booking_date_m,booking_date_d,booking_date_w,checkin_date_y,checkin_date_m,checkin_date_d,checkin_date_w,checkout_date_y,checkout_date_m,checkout_date_d,checkout_date_w,family_type,NumberOfPeople,resort_region_type_code,state_code_residence_resort,resort_type_cluster_code
341424,1,3,4,2,7,2,0,45,0,3,3,4,1,2.0,2.0,3,3,1,2018,4,5,14,2018,4,5,14,2018,4,6,14,0,2,6,5.0,7
341425,1,1,0,2,7,4,0,45,1,1,1,4,8,2.0,2.0,4,2,4,2015,5,24,21,2015,6,23,26,2015,6,27,26,0,4,2,6.0,1
341426,1,3,2,2,7,3,0,45,1,2,2,2,3,4.0,2.0,2,3,3,2015,7,21,30,2015,8,7,32,2015,8,10,33,0,3,4,4.0,4
341427,1,1,0,2,7,3,0,45,1,1,1,2,3,2.0,2.0,6,3,3,2015,7,31,31,2015,10,22,43,2015,10,25,43,0,3,2,8.0,1
341428,1,1,0,2,7,4,0,45,1,1,2,3,8,2.0,2.0,11,2,4,2016,7,1,26,2016,10,1,39,2016,10,5,40,0,4,3,13.0,2


In [328]:
sample_submission=pd.read_csv('sample_submission.csv')

In [329]:
s=pd.DataFrame({'reservation_id':sample_submission['reservation_id'],'amount_spent_per_room_night_scaled':cb_model.predict(dftest)})
s.head()


s.to_csv('Submission1.csv',index=False)

In [331]:
cb_model.predict(dftest)

array([7.9047469 , 7.37856509, 7.51383464, ..., 8.34739619, 7.96104928,
       7.22232088])

In [308]:
mean_squared_error?

In [307]:
from sklearn.metrics import mean_squared_error


In [312]:
from sklearn.utils.multiclass import type_of_target
type_of_target(y_train)



'continuous'

In [ ]:
store evey distribution in seperate list?

In [319]:
# #20 fold cv
# from sklearn.metrics import mean_squared_error
# errcb=[]
# y_pred_totcb=[]
# from sklearn.model_selection import KFold,StratifiedKFold
# fold=StratifiedKFold(n_splits=20,shuffle=True,random_state=100)
# i=1
# for train_index, test_index in fold.split(X_train,y_train):
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y[train_index], y[test_index]
#     m=catboost.CatBoostRegressor(iterations=1000,
#                              learning_rate=0.1,
#                              depth=2,
#                              eval_metric='RMSE',
#                              random_seed = 23,
#                              bagging_temperature = 0.2,
#                              od_type='Iter',
#                              metric_period = 75,
#                              od_wait=100)
#     m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
#     preds=m.predict_proba(dftest.iloc[:,1:])[:,-1]
#     print("err: ",roc_auc_score(y_test,preds))
#     errcb.append(roc_auc_score(y_test,preds))
#     p = m.predict_proba(dftest)[:,-1]
#     y_pred_totcb.append(p)

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, random_state=None)
# X is the feature set and y is the target
for train_index, test_index in skf.split(X,y): 
    print("Train:", train_index, "Validation:", val_index) 
    X_train, X_test = X[train_index], X[val_index] 
    y_train, y_test = y[train_index], y[val_index]


In [320]:
model = catboost.CatBoostRegressor(iterations=250,
                          learning_rate=0.92,
                          depth=2)
# Fit model
model.fit(X_train, y_train)
# Get predictions
preds = model.predict(X_val)


NameError: name 'catboost' is not defined

In [272]:
cb_model.predict(dftest.iloc[:,1:])


array([7.94520016, 7.3824843 , 7.51307392, ..., 8.36579751, 8.46236153,
       7.00349149])

In [181]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

In [73]:
sample_submission=pd.read_csv('sample_submission.csv')

In [74]:
sample_submission.head()

,reservation_id,amount_spent_per_room_night_scaled
0,7dae1ce6bc8f69481328f2be5c4943077dad5598b5f66d...,8
1,fe0d4e444e1818436c88f72f1cf800536c2f785e59baeb...,8
2,540bd4285ad8168e8388f84ee74a82cd4f97dc0a404d3e...,8
3,09593c907ae262e46f655b4db9e14f54a19eadcfdd2679...,8
4,f4c50caac68051faf37551d70bb17eebef2a20e2244cb1...,8


In [258]:
s=pd.DataFrame({'reservation_id':sample_submission['reservation_id'],'amount_spent_per_room_night_scaled':model.predict(dftest.iloc[:,1:])})
s.head()


s.to_csv('Submission1.csv',index=False)

In [225]:
 $conda search —all scikit-learn

SyntaxError: invalid syntax (<ipython-input-225-5409aafc9367>, line 1)

In [235]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC  
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import  BaggingRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingClassifier


In [240]:
BaggingRegressor?

In [246]:
X_train.shape

(273139, 35)

In [247]:
y_train.shape

(273139,)

In [162]:
from sklearn.feature_selection import RFE
#create the RFE model and select 3 attributes
rfcr = CatBoostRegressor(iterations=1000,
                             learning_rate=0.9,
                             depth=2,
                             eval_metric='RMSE',
                             random_seed = 23,
                             loss_function='RMSE',
                             bagging_temperature =0.2,
                             od_type='Iter',
                             metric_period = 75,
                             od_wait=100)
clf_LR = rfcr
clf_LR.fit(X_train,y_train)
rfe = RFE(clf_LR, 10)
rfe = rfe.fit(X_train,y_train)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

0:	learn: 1.3283773	total: 23.7ms	remaining: 23.7s
75:	learn: 0.9824924	total: 1.81s	remaining: 22s
150:	learn: 0.9789983	total: 3.58s	remaining: 20.1s
225:	learn: 0.9769369	total: 5.32s	remaining: 18.2s
300:	learn: 0.9754277	total: 7.06s	remaining: 16.4s
375:	learn: 0.9741601	total: 8.81s	remaining: 14.6s
450:	learn: 0.9729564	total: 10.6s	remaining: 12.9s
525:	learn: 0.9719681	total: 12.3s	remaining: 11.1s
600:	learn: 0.9710357	total: 14s	remaining: 9.32s
675:	learn: 0.9701494	total: 15.8s	remaining: 7.56s
750:	learn: 0.9693929	total: 17.5s	remaining: 5.81s
825:	learn: 0.9686567	total: 19.2s	remaining: 4.05s
900:	learn: 0.9679577	total: 21s	remaining: 2.3s
975:	learn: 0.9672320	total: 22.7s	remaining: 559ms
999:	learn: 0.9670613	total: 23.3s	remaining: 0us
0:	learn: 1.3283773	total: 26.2ms	remaining: 26.2s
75:	learn: 0.9824924	total: 1.87s	remaining: 22.7s
150:	learn: 0.9789983	total: 3.68s	remaining: 20.7s
225:	learn: 0.9769369	total: 5.53s	remaining: 18.9s
300:	learn: 0.9754277	tot

750:	learn: 0.9699651	total: 17.9s	remaining: 5.94s
825:	learn: 0.9692862	total: 19.7s	remaining: 4.15s
900:	learn: 0.9686399	total: 21.5s	remaining: 2.36s
975:	learn: 0.9679724	total: 23.3s	remaining: 573ms
999:	learn: 0.9677885	total: 23.9s	remaining: 0us
0:	learn: 1.3352492	total: 25.7ms	remaining: 25.7s
75:	learn: 0.9831136	total: 1.75s	remaining: 21.3s
150:	learn: 0.9799311	total: 3.55s	remaining: 20s
225:	learn: 0.9777186	total: 5.34s	remaining: 18.3s
300:	learn: 0.9762060	total: 7.12s	remaining: 16.5s
375:	learn: 0.9749285	total: 8.91s	remaining: 14.8s
450:	learn: 0.9739389	total: 10.7s	remaining: 13s
525:	learn: 0.9729124	total: 12.5s	remaining: 11.3s
600:	learn: 0.9718519	total: 14.3s	remaining: 9.48s
675:	learn: 0.9710187	total: 16.1s	remaining: 7.71s
750:	learn: 0.9702213	total: 17.9s	remaining: 5.93s
825:	learn: 0.9695073	total: 19.7s	remaining: 4.15s
900:	learn: 0.9688403	total: 21.6s	remaining: 2.37s
975:	learn: 0.9682498	total: 23.4s	remaining: 574ms
999:	learn: 0.968058

375:	learn: 0.9769109	total: 8.25s	remaining: 13.7s
450:	learn: 0.9759556	total: 9.92s	remaining: 12.1s
525:	learn: 0.9751717	total: 11.6s	remaining: 10.4s
600:	learn: 0.9744289	total: 13.3s	remaining: 8.83s
675:	learn: 0.9737793	total: 15s	remaining: 7.17s
750:	learn: 0.9730756	total: 16.7s	remaining: 5.53s
825:	learn: 0.9725235	total: 18.3s	remaining: 3.87s
900:	learn: 0.9719647	total: 20s	remaining: 2.19s
975:	learn: 0.9714502	total: 21.6s	remaining: 532ms
999:	learn: 0.9712908	total: 22.2s	remaining: 0us
0:	learn: 1.3360851	total: 19.7ms	remaining: 19.7s
75:	learn: 0.9848339	total: 1.58s	remaining: 19.3s
150:	learn: 0.9810783	total: 3.21s	remaining: 18s
225:	learn: 0.9794099	total: 4.83s	remaining: 16.5s
300:	learn: 0.9780074	total: 6.46s	remaining: 15s
375:	learn: 0.9769174	total: 8.09s	remaining: 13.4s
450:	learn: 0.9759541	total: 9.71s	remaining: 11.8s
525:	learn: 0.9751651	total: 11.3s	remaining: 10.2s
600:	learn: 0.9744026	total: 13s	remaining: 8.6s
675:	learn: 0.9737926	total

In [163]:
rfe.predict(dftest.iloc[:,1:])

array([7.95874176, 7.37044594, 7.66716177, ..., 8.46381676, 8.19501055,
       7.27681844])

In [164]:
s=pd.DataFrame({'reservation_id':sample_submission['reservation_id'],'amount_spent_per_room_night_scaled':rfe.predict(dftest.iloc[:,1:])})
s.head()


s.to_csv('Submission1.csv',index=False)

### Stacking Models

In [320]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb


In [321]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    rmse= np.sqrt(-cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [322]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_ctboost = catboost.CatBoostRegressor(iterations=100,
                          learning_rate=0.92,
                          depth=2)
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [329]:
y_train.shape

(273139,)

In [330]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X_train, y_train)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

In [340]:
averaged_models = AveragingModels(models = (model_ctboost, model_lgb, lasso))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

SyntaxError: invalid syntax (<ipython-input-340-0b470b9efeb5>, line 2)

In [341]:
averaged_models.predict(dftest.iloc[:,1:])

AttributeError: 'AveragingModels' object has no attribute 'models_'

In [389]:
X=X_train
y=y_train

In [390]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [393]:
stacked_averaged_models = StackingAveragedModels(base_models = (model_lgb,model_xgb),
                                                 meta_model = lasso)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [386]:
X_train.shape

(238996, 35)

In [387]:
stacked_averaged_models.fit(X_train, y_train)
# stacked_train_pred = stacked_averaged_models.predict(X_train.values)
# stacked_pred = np.expm1(stacked_averaged_models.predict(dftest.iloc[:,1:]))
# print(rmsle(y_train, stacked_train_pred))

KeyError: "None of [Int64Index([     0,      1,      2,      3,      4,      5,      6,      8,\n                 9,     11,\n            ...\n            238984, 238985, 238986, 238987, 238988, 238990, 238992, 238993,\n            238994, 238995],\n           dtype='int64', length=191196)] are in the [columns]"

In [363]:
y_valid.shape

(68285,)

In [365]:
X_train.dtypes

booking_type_code              category
channel_code                   category
cluster_code                   category
main_product_code              category
member_age_buckets                int64
numberofadults                    int64
numberofchildren                  int64
persontravellingid             category
reservationstatusid_code       category
resort_region_code             category
resort_type_code               category
room_type_booked_code          category
roomnights                        int64
season_holidayed_code          category
state_code_residence           category
state_code_resort              category
total_pax                         int64
Stay_duration                     int64
booking_date_y                    int64
booking_date_m                    int64
booking_date_d                    int64
booking_date_w                    int64
checkin_date_y                    int64
checkin_date_m                    int64
checkin_date_d                    int64


In [366]:
df[['family_type', 'persontravellingid', 'member_age_buckets', 'cluster_code','reservationstatusid_code','booking_type_code',
        'state_code_resort','state_code_residence','season_holidayed_code','room_type_booked_code','resort_type_code',
       'resort_region_code','main_product_code','channel_code','NumberOfPeople','resort_region_type_code','state_code_residence_resort','resort_type_cluster_code','member_age_buckets']] = df[['family_type', 'persontravellingid', 'member_age_buckets', 'cluster_code','reservationstatusid_code','booking_type_code',
        'state_code_resort','state_code_residence','season_holidayed_code','room_type_booked_code','resort_type_code',
       'resort_region_code','main_product_code','channel_code','NumberOfPeople','resort_region_type_code','state_code_residence_resort','resort_type_cluster_code','member_age_buckets']].astype('int')


In [370]:
X_train.dtypes

booking_type_code              int32
channel_code                   int32
cluster_code                   int32
main_product_code              int32
member_age_buckets             int32
numberofadults                 int64
numberofchildren               int64
persontravellingid             int32
reservationstatusid_code       int32
resort_region_code             int32
resort_type_code               int32
room_type_booked_code          int32
roomnights                     int64
season_holidayed_code          int32
state_code_residence           int32
state_code_resort              int32
total_pax                      int64
Stay_duration                  int64
booking_date_y                 int64
booking_date_m                 int64
booking_date_d                 int64
booking_date_w                 int64
checkin_date_y                 int64
checkin_date_m                 int64
checkin_date_d                 int64
checkin_date_w                 int64
checkout_date_y                int64
c

In [372]:
model_xgb.predict(dftest.iloc[:,1:])

array([7.9540277, 7.4737735, 7.5100455, ..., 8.425686 , 8.406819 ,
       7.2673573], dtype=float32)

In [379]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    rmse= np.sqrt(-cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [395]:
cb_model.fit(X_train, y_train)
xgb_train_pred = cb_model.predict(X_train)
xgb_pred = np.expm1(cb_model.predict(dftest.iloc[:,1:]))
print(rmsle_cv(cb_model))

0:	learn: 7.0334690	total: 89.7ms	remaining: 1m 29s
75:	learn: 1.0101427	total: 7.09s	remaining: 1m 26s
150:	learn: 0.9986010	total: 15.1s	remaining: 1m 24s
225:	learn: 0.9937701	total: 22.7s	remaining: 1m 17s
300:	learn: 0.9914228	total: 31s	remaining: 1m 11s
375:	learn: 0.9898874	total: 39.7s	remaining: 1m 5s
450:	learn: 0.9888236	total: 47.2s	remaining: 57.5s
525:	learn: 0.9879499	total: 56.8s	remaining: 51.2s
600:	learn: 0.9871927	total: 1m 4s	remaining: 42.8s
675:	learn: 0.9865801	total: 1m 11s	remaining: 34.1s
750:	learn: 0.9860227	total: 1m 19s	remaining: 26.5s
825:	learn: 0.9855279	total: 1m 26s	remaining: 18.3s
900:	learn: 0.9850772	total: 1m 33s	remaining: 10.3s
975:	learn: 0.9847068	total: 1m 41s	remaining: 2.5s
999:	learn: 0.9846005	total: 1m 44s	remaining: 0us
0:	learn: 7.0332184	total: 78.9ms	remaining: 1m 18s
75:	learn: 1.0094047	total: 6.85s	remaining: 1m 23s
150:	learn: 0.9980237	total: 13.2s	remaining: 1m 14s
225:	learn: 0.9936185	total: 21.6s	remaining: 1m 13s
300:	l